In [1]:
import tensorflow as tf

def train():
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    print("Training...")

    model.fit(
        x_train, y_train, 
        epochs=3, 
        validation_split=0.2 
    ) 

    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

In [2]:
import os
from kubeflow import fairing

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

fairing.config.set_builder(
    'append',
    base_image='tensorflow/tensorflow:latest-py3',
#   base_image=f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image
    registry = PRIVATE_REGISTRY, 
    push=True
)

fairing.config.set_deployer('job')

remote_train = fairing.config.fn(train)
remote_train()

[I 201206 00:59:46 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f6384195b00>
[I 201206 00:59:46 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f6384195b70>
[I 201206 00:59:46 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f6384195be0>
[W 201206 00:59:46 append:50] Building image using Append builder...
[I 201206 00:59:46 base:107] Creating docker context: /tmp/fairing_context_19ctamnz
[W 201206 00:59:46 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201206 00:59:46 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:latest-py3'
[W 201206 00:59:48 append:54] Image successfully built in 2.4838827240018873s.
[W 201206 00:59:48 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/fairing-job:956A7281...
[I 201206 00:

2020-12-06 00:59:52.411611: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-12-06 00:59:52.411864: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-12-06 00:59:52.411967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "/app/function_shim.py", line 79, in <module>
    call(args.serialized_fn_file)
  File "/app/function_shim.py", line 64, in call
    res = obj()
  File "<ipython-input-1-0dbc804d7d9a>", line 4, in train
AttributeError: module 'tensorflow' 

[W 201206 00:59:54 job:173] Cleaning up job fairing-job-bnr85...


### curl로 프라이빗 레지스트리 확인

In [3]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/_catalog

{"repositories":["02-python-file-fairing","covid19-katib-job","fairing-job","katib-base","katib-job","kf-base","my-02-faring-job","my-02-python-file-fairing","my-03-notebook-fairing-job","my-04-notebook-single-file-fairing-job","my-06-build-only-fairing","my-fairing-job","my-notebook-fairing-job","tensorboard-job"]}


In [4]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/fairing-job/tags/list

{"name":"fairing-job","tags":["3DA1A245","42CFDC19","37453BCC","910F73B3","18DF7E85","F5D207F","E69B7B0","795AC9AC","223E3522","141D2F60","A5EE9B45","956A7281","8E296311","66787D3","F60C4742","3052981C"]}
